# Поиск токсичных комментариев пользователей интернет-магазина

Интернет-магазину нужен инструмент, который будет искать токсичные комментарии пользователей и отправлять их на модерацию. 

Требуется обучить модель для классификации комментариев на позитивные и негативные. Имеется набор данных с разметкой о токсичности правок.

Необходимо построить модель со значением метрики качества *F1* не меньше 0.75. 


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем необходимые библиотеки и инструменты

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from tqdm import notebook
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Загрузим данные и ознакомимся с ними

In [2]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
    
except:
    df = pd.read_csv('toxic_comments.csv') # для локальной работы

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
df['toxic'].value_counts(normalize=1)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

- Пропусков в данных нет
- Около 10% токсичных комментариев

Приведём все комментарии к нижнему регистру

In [6]:
df['text'] = df['text'].str.lower()

Сохраним корпус текстов и загрузим лемматизатор

In [7]:
corpus = list(df['text'])

lemmatizer = WordNetLemmatizer()

Создадим функции для лемматизации каждого слова с его POS-тегом

In [8]:
# Функция POS_TAGGER

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

# Функция лемматизации текста
    
def lemmatize(text):
    
    # Токенизация текста и нахождение POS-тега для каждого токена
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(text))

    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))

    lemmatized_text = []
    for word, tag in wordnet_tagged:
        if tag is None:
        # если тег не доступен, добавляем токен как есть
            lemmatized_text.append(word)
        else:
    # иначе используем тег для лемматизации токена
            lemmatized_text.append(lemmatizer.lemmatize(word, tag))
    lemmatized_text = " ".join(lemmatized_text)
    
    return lemmatized_text

Создадим функцию для очистки комментариев

In [9]:
# Функция очистки текствов (оставляет только строчные буквы и пробелы)

def clear_text(text):

    sub_text = re.sub(r'[^a-z ]', ' ', text)
    split_text = sub_text.split()
    join_text = " ".join(split_text)
    return join_text

Лемматизируем наш корпус

In [10]:
lemm_corpus = []

for i in notebook.tqdm(range(len(corpus))):
    
    lemm_corpus.append(lemmatize(clear_text(corpus[i])))

Сохраним лемматизированный корпус в исходном датасете

In [11]:
df['lemm_text'] = lemm_corpus

Разделим исходный датасет на обучающий и тестовый в соотношении 75:25 таким образом, чтобы процент токсичных комментариев в датасетах был одинаковым

In [12]:
df_train, df_test = train_test_split(df, test_size=0.25, stratify=df['toxic'])

Проверим, что процент токсичных комментариев в обучающем и тестовом датасетах одинаков

In [13]:
df_train['toxic'].value_counts(normalize=1)

0    0.898319
1    0.101681
Name: toxic, dtype: float64

In [14]:
df_test['toxic'].value_counts(normalize=1)

0    0.898328
1    0.101672
Name: toxic, dtype: float64

Загрузим стоп-слова

In [15]:
stopwords = set(nltk_stopwords.words('english'))

Сохраним преобразованные обучающий и тестовый корпуса

In [16]:
corpus_train = df_train['lemm_text'].values.astype('U')

corpus_test = df_test['lemm_text'].values.astype('U')

Рассчитаем величины TF-IDF для обучающего и тестового корпусов

In [17]:
tf_idf = TfidfVectorizer(stop_words=stopwords) # Создадим счётчик со стоп-словами

tf_idf_train = tf_idf.fit_transform(corpus_train)

tf_idf_test = tf_idf.transform(corpus_test)

## Обучение

Создадим функцию для обучения и вычисления метрик модели

In [18]:
def fit_predict(model, features_train, target_train, features_test, target_test):
    
    
    model.fit(features_train, target_train)

    predictions = model.predict(features_test)

    f1 = round(f1_score(target_test, predictions), 2)

    recall = round(recall_score(target_test, predictions), 2)

    precision = round(precision_score(target_test, predictions), 2)
    
    return f1, recall, precision

Обучим модель LogisticRegression и рассчитаем её метрики для тестовой выборки

In [19]:
f1_lr, recall_lr, precision_lr = fit_predict(LogisticRegression(), tf_idf_train, df_train['toxic'], tf_idf_test, df_test['toxic'])

print('F1-метрика для LogisticRegression:', f1_lr)
print()
print('Полнота для LogisticRegression:', recall_lr)
print()
print('Точность для LogisticRegression:', precision_lr)

F1-метрика для LogisticRegression: 0.74

Полнота для LogisticRegression: 0.62

Точность для LogisticRegression: 0.92


Обучим модель LinearSVC и рассчитаем её метрики для тестовой выборки

In [20]:
f1_lsvc, recall_lsvc, precision_lsvc = fit_predict(LinearSVC(random_state=12345), tf_idf_train, df_train['toxic'], tf_idf_test, df_test['toxic'])

print('F1-метрика для LinearSVC:', f1_lsvc)
print()
print('Полнота для LinearSVC:', recall_lsvc)
print()
print('Точность для LinearSVC:', precision_lsvc)

F1-метрика для LinearSVC: 0.78

Полнота для LinearSVC: 0.7

Точность для LinearSVC: 0.88


Обучим модель DummyClassifier и рассчитаем её метрики для тестовой выборки

In [21]:
f1_dc, recall_dc, precision_dc = fit_predict(DummyClassifier(strategy='stratified', random_state=12345), tf_idf_train, df_train['toxic'], tf_idf_test, df_test['toxic'])

print('F1-метрика для DummyClassifier:', f1_dc)
print()
print('Полнота для DummyClassifier:', recall_dc)
print()
print('Точность для DummyClassifier:', precision_dc)

F1-метрика для DummyClassifier: 0.11

Полнота для DummyClassifier: 0.11

Точность для DummyClassifier: 0.11


## Выводы

Создадим итоговую таблицу для моделей

In [22]:
final = pd.DataFrame([['LogisticRegression', f1_lr, recall_lr, precision_lr],
                  ['LinearSVC', f1_lsvc, recall_lsvc, precision_lsvc],
                  ['DummyClassifier', f1_dc, recall_dc, precision_dc]],
                 columns = ['Название модели', 'F1-метрика', 'Полнота', 'Точность'])
final

,Название модели,F1-метрика,Полнота,Точность
0,LogisticRegression,0.74,0.62,0.92
1,LinearSVC,0.78,0.70,0.88
2,DummyClassifier,0.11,0.11,0.11


- Модель LinearSVC превосходит требуемый минимальный уровень качества
- Модели LinearSVC и LogisticRegression проходят тест на адекватность
- Модель LogisticRegression немного точнее определяет токсичные комментарии, но выявляет их меньший процент